In [89]:
%run data_cleaning.py


In [90]:
# #random forest regression model for tumour size
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split
# import numpy as np
# np.random.seed(42)

# #Input parameters = everything except survival months and status
# # data1 = remove_outliers_tumor_size(df)
# # data1.drop("Status", axis=1, inplace=True)


# x = data1.drop(["Survival Months", "Tumor Size", "6th Stage", "T Stage", "Regional Node Examined"], axis=1)
# y = data1["Tumor Size"]

# x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2)
# x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=1)
# # Use RandomForestRegressor for regression tasks

# rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=8000).fit(x_train, y_train)

# # Making predictions based on the model
# y_pred = rf.predict(x_test)
# y_val_pred = rf.predict(x_val)


# #Checking accuracy on validation set
# mse_validation = mean_squared_error(y_val, y_val_pred)
# print("Mean Squared Error on Validation Set:", mse_validation,"(((", str(mse_validation**0.5) + "))")

# y_test_pred = rf.predict(x_test)
# #Checking accuracy on test set
# mse_test = mean_squared_error(y_test, y_test_pred)
# print("Mean Squared Error on Test Set:", str(mse_test) + " ((" + str(mse_test**0.5) + "))")

# # Getting feature importances
# feature_importances = pd.DataFrame({'Feature': x.columns, 'Importance': rf.feature_importances_})

# # Sorting features by importance in descending order
# feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# # Print or display the feature importances
# print("Feature Importances:")
# print(feature_importances)


# #original: V:34.62, T: 34.94
# #only using suitable data: V:84.47, T: 84.88 
# #tree depth = 3: V: 83.53, T:84.349

In [1]:
#random forest regression model for tumour size
###NO VALIDATION SET
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(42)

#Input parameters = everything except survival months and status
# data1 = remove_outliers_tumor_size(df)
# data1.drop("Status", axis=1, inplace=True)
data1 = remove_outliers_tumor_size(df)

x = data1.drop(["Survival Months", "Tumor Size", "6th Stage", "T Stage", "Regional Node Examined"], axis=1)
y = data1["Tumor Size"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
# Use RandomForestRegressor for regression tasks
rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=8000).fit(x_train, y_train)

# Making predictions based on the model
y_pred = rf.predict(x_test)



y_test_pred = rf.predict(x_test)
#Checking accuracy on test set
mse_test = mean_squared_error(y_test, y_test_pred)
print("Mean Squared Error on Test Set:", str(mse_test) + " ((" + str(mse_test**0.5) + "))")

# Getting feature importances
feature_importances = pd.DataFrame({'Feature': x.columns, 'Importance': rf.feature_importances_})

# Sorting features by importance in descending order
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Print or display the feature importances
print("Feature Importances:")
print(feature_importances)


#original: V:34.62, T: 34.94
#only using suitable data: V:84.47, T: 84.88 
#tree depth = 3: V: 83.53, T:84.349

NameError: name 'remove_outliers_tumor_size' is not defined

In [92]:
#random forest regression model for Suriaval months (only for dead people)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(42)

#Input parameters = everything except survival months
data2 = df.copy()
data2 = data2[data2["Status"] == 0]
data2 = remove_outliers_survival_months(data2)


x = data2[['Estrogen Status', 'Progesterone Status', '6th Stage', 'Age']]
y = data2["Survival Months"]

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5)
# Use RandomForestRegressor for regression tasks

rf = RandomForestRegressor(random_state=42, n_estimators=1000, max_depth=5).fit(x_train, y_train)

# Making predictions based on the model
y_pred = rf.predict(x_test)
y_val_pred = rf.predict(x_val)


#Checking accuracy on validation set
mse_validation = mean_squared_error(y_val, y_val_pred)
print("Mean Squared Error on Validation Set:", mse_validation,"(((", str(mse_validation**0.5) + "))")

y_test_pred = rf.predict(x_test)
#Checking accuracy on test set
mse_test = mean_squared_error(y_test, y_test_pred)
print("Mean Squared Error on Test Set:", str(mse_test) + " ((" + str(mse_test**0.5) + "))")

# Getting feature importances
feature_importances = pd.DataFrame({'Feature': x.columns, 'Importance': rf.feature_importances_})

# Sorting features by importance in descending order
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Print or display the feature importances
print("Feature Importances:")
print(feature_importances)


#original: V:276.417, T:267.399



Mean Squared Error on Validation Set: 279.38318566237393 ((( 16.714759515541164))
Mean Squared Error on Test Set: 264.8786045708709 ((16.275091538018177))
Feature Importances:
               Feature  Importance
3                  Age    0.576221
2            6th Stage    0.201637
1  Progesterone Status    0.127068
0      Estrogen Status    0.095074
